In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
import missingno as msno
import pdb
from scipy import stats
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Read in Pickle

In [2]:
cleaned_KC = pd.read_pickle('./data/cleaned_KC.pkl')

## Convert Features to Categorical

In [3]:
categorical_features = ['bathrooms','bedrooms','zipcode','yr_built','grade','condition','view','floors']

In [4]:
# converts all categorical features to a categorical type
for feature in categorical_features:
    cleaned_KC[feature] = cleaned_KC[feature].astype('category')
cleaned_KC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21347
Data columns (total 15 columns):
id               21301 non-null int64
date             21301 non-null object
price            21301 non-null float64
bedrooms         21301 non-null category
bathrooms        21301 non-null category
sqft_living      21301 non-null int64
sqft_lot         21301 non-null int64
floors           21301 non-null category
view             21301 non-null category
condition        21301 non-null category
grade            21301 non-null category
yr_built         21301 non-null category
zipcode          21301 non-null category
sqft_living15    21301 non-null int64
sqft_lot15       21301 non-null int64
dtypes: category(8), float64(1), int64(5), object(1)
memory usage: 1.5+ MB


In [5]:
cleaned_KC.describe()

,id,price,sqft_living,sqft_lot,sqft_living15,sqft_lot15
count,2.130100e+04,2.130100e+04,21301.000000,2.130100e+04,21301.000000,21301.000000
mean,4.588552e+09,5.290905e+05,2050.985869,1.489689e+04,1975.873527,12627.651753
std,2.877289e+09,3.295859e+05,861.676760,4.078791e+04,672.280276,26526.526949
min,1.000102e+06,7.800000e+04,370.000000,5.200000e+02,399.000000,651.000000
25%,2.125049e+09,3.200000e+05,1420.000000,5.029000e+03,1480.000000,5100.000000
50%,3.905010e+09,4.500000e+05,1900.000000,7.590000e+03,1830.000000,7611.000000
75%,7.331900e+09,6.370000e+05,2520.000000,1.058400e+04,2350.000000,10042.000000
max,9.900000e+09,4.490000e+06,7850.000000,1.651359e+06,6210.000000,871200.000000


## Review Correlation and VIF 

#### Calculate Pearson's R Value

In [6]:
cleaned_KC.corr()['price'].sort_values(ascending=False)

price            1.000000
sqft_living      0.677350
sqft_living15    0.593302
sqft_lot         0.083969
sqft_lot15       0.077855
id              -0.015326
Name: price, dtype: float64

#### Calculate VIF

Based on the first run of the VIF, Sqft_living and sqft_living15 are thought to have multi-collinearity, thus we will drop sqft_living15

In [7]:
# select only columns with datatype int64 or float 64
numeric_cleaned_KC = cleaned_KC.select_dtypes(include=['int64','float64'])

# then drop the sqft_living15 and id column to reduce noise.
numeric_cleaned_KC = numeric_cleaned_KC.drop(axis=1, columns=['sqft_living15', 'id'])

vif = pd.DataFrame()

# For each column,run a variance_inflaction_factor against all other columns to get a VIF Factor score
vif["VIF Factor"] = [variance_inflation_factor(numeric_cleaned_KC.values, i) for i in range(numeric_cleaned_KC.shape[1])]

# simply label the scores with their related columns
vif["features"] = numeric_cleaned_KC.columns

vif.round(1)

,VIF Factor,features
0,6.6,price
1,7.0,sqft_living
2,2.3,sqft_lot
3,2.5,sqft_lot15


## Linear Regression
##### Price vs Sqft_living

#### Calculate Regression Line

In [ ]:
# Transforms a dataframe into a np.array
X = cleaned_KC['sqft_living'].values.reshape(-1,1)
y = cleaned_KC['price'].values.reshape(-1,1)

# Create a Linear Regression object and fit a line using X and y
reg = LinearRegression()
reg.fit(X, y)

# using that line, make predictions for y using X
predictions = reg.predict(X)

#### Plot Regression against Scatterplot

In [ ]:
# Look into Seaborns scatterplot for a new look

In [ ]:
# Make a 16x8 figure that has...
plt.figure(figsize=(16, 8))

# ...a scatter plot of sqft_living vs price...
plt.scatter(
    cleaned_KC['sqft_living'],
    cleaned_KC['price'],
    c='black'
)

# ...plot the regression line...
plt.plot(
    cleaned_KC['sqft_living'],
    predictions,
    c='blue',
    linewidth=1
)

# ...set the labels and show
plt.xlabel("Sqft. Living")
plt.ylabel("Price)")
plt.show()

#### Calculate OLS Regression Results

In [ ]:
X = cleaned_KC['sqft_living']
y = cleaned_KC['price']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2).fit()
print(est.summary())

### <span style="color:blue">Our Linear Regression R-Squared result:</span><span style="color:red"> .52</span>

## Multiple Linear Regression including Categorical Variables
### Run Train/Test split

#### convert to dummy variables dataframe

In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

# target = titanic2.survived.values
# features = titanic2[['pclass', 'sex', 'age', 'fare', 'embarked']]

# This dataset contains some categorical variables ("pclass", "sex" and "embarked")
categorial_columns = ['bathrooms','bedrooms','grade','view','floors']

# numerical variables ("age" and "fare").
numerical_columns = ['price','sqft_lot','sqft_living15', 'sqft_lot15']

preprocess = make_column_transformer(
    (numerical_columns, StandardScaler()),
    (categorial_columns, OneHotEncoder())
)

test_array = preprocess.fit_transform(cleaned_KC).toarray()

test_df = pd.DataFrame(test_array)
test_df.head()

/usr/local/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:778: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,-0.932072,-0.226712,-0.945868,-0.263050,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.027033,-0.187680,-0.425240,-0.188067,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-1.059204,-0.120060,1.106895,-0.172120,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.227289,-0.242648,-0.916118,-0.287555,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.057924,-0.167134,-0.261614,-0.193194,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
categorial_columns = ['bathrooms','bedrooms','grade','view','floors']
non_categorial_columns = ['id','date','price','sqft_lot','condition','yr_built','zipcode','sqft_living15', 'sqft_lot15']

# This replaces categorical columns with dummy variables, and drops non categorical columns
dummies_df = pd.get_dummies(cleaned_KC, columns=categorial_columns).drop(columns=non_categorial_columns)

#### Create X & Y Train/Test variables

In [10]:
# This sets X to a dataframe consisting of columns with dummy variables & without extra columns.
X = dummies_df

# this simply sets y to the price column of the dataframe
y = cleaned_KC['price']

# This runs a train/test split with .20 of the data. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#### Calculate OLS

In [ ]:
# Simply adding a y-intercept column.
X2 = sm.add_constant(X_train)

# Using statsmodels, run an Ordinary Least Squares test using the array of y values, and the dataframe of x_test values and fit a line to it. 
est = sm.OLS(y_train, X2).fit()
print(est.summary())

#### Calculate skLearn r2

In [15]:
# Creates a Linear Regression object
lin_reg = LinearRegression()

# Train the Regression object to fit a line with train data
lin_reg.fit(X_train, y_train)

# using that line, predict y values against x_test values.
y_pred = lin_reg.predict(X_test)

# Use sklearn.metrics.r2_score to determine r-squared based on y_test values and y_pred values.
r2_score(y_test, y_pred)

0.6193775664071248

### <span style="color:blue">Our Multiple Linear Regression R-Squared result:</span><span style="color:green"> .62</span>

#### Calculate skLearn MSE

In [16]:
mean_squared_error(y_test, y_pred)

42720018186.52437

# Lasso/Ridge

In [11]:
from sklearn.model_selection import GridSearchCV

In [17]:
lasso = Lasso()
ridge = Ridge()

parameters = {'alpha': [1e-15,1e-10,1e-8,1e-4,1e-3,1e-2,1,5,10,20]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv = 5)

ridge_regressor.fit(X,y)

print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.6096e-21): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.55513e-21): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.38561e-21): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.34126e-21): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.50913e-21): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarnin

{'alpha': 1}
-41990606064.27571


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [18]:
lasso = Lasso()

parameters = {'alpha': [1e-15,1e-10,1e-8,1e-4,1e-3,1e-2,1,5,10,20]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv = 5)

lasso_regressor.fit(X,y)

print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 351270416070819.3, tolerance: 183954266615.8631
  positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 353265734384331.6, tolerance: 187423008930.7734
  positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 355965893711343.7, tolerance: 188902175112.9633
  positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 351703969251683.

{'alpha': 20}
-42015458146.942635
